### await

Suspend the execution of coroutine on an awaitable. 
- An awaitable object can be a coroutine or an object with `__await__` method.
    - `__await__` method must return an iterator (usually a generator iterator).
- Can only be used inside a coroutine function.
- Behavior depends on the type of awaitable:
    - Task 
        - Cede control (B)
        - Additionally, in general, ensure such that:
            - When awaited task finishes, the original task or coro is added back to the event loop to-do list.
            - For example:
                - The await instruction adds a callback to the original task/coroutine (A) to the task's callback list (B)
                - When the awaited task is picked and finishes, add the callback to the event loop's to-do list
    - Coroutine
        - Does not cede control (Z), behavior same as normal funcs
        - Note this can be a way for asyncio to differentiate between task that need or not need CPU time.
    - Future
        - The coroutine will wait until the Future is resolved in some other place.

In [2]:
import asyncio

In [11]:
async def dig_the_hole():
    print("Digging the hole...")


async def plant_a_tree():
    dig_the_hole_task = asyncio.create_task(dig_the_hole())
    await dig_the_hole_task  # B
    # A: resume point in the call back

    print("Other instructions associated with planting a tree ..")


await asyncio.gather(
    plant_a_tree(),
    plant_a_tree(),
)

Digging the hole...
Digging the hole...
Other instructions associated with planting a tree ..
Other instructions associated with planting a tree ..


[None, None]

In [10]:
import asyncio


async def coro_a():
    #    await asyncio.sleep(0) # fix by awaiting another awaitable that yields control
    print("I am coro_a(). Hi!")


async def coro_b():
    print("I am coro_b(). I sure hope no one hogs the event loop...")


async def main():
    task_b = asyncio.create_task(coro_b())
    num_repeats = 2
    for _ in range(num_repeats):
        await coro_a()  # Z: control never ceded to event loop, so task_b can not start
    await task_b


await main()

I am coro_a(). Hi!
I am coro_a(). Hi!
I am coro_b(). I sure hope no one hogs the event loop...
